In [1]:
#todo: testing, translation, clean up use of graph data structure

In [2]:
### graph data structure
class graph:
    def __init__(self, vertexlist, edgelist):
        self.vertices = vertexlist
        self.edges = edgelist
        self.neighbors = {v: [] for v in vertexlist}
        self.leaves = vertexlist
        for (u,v) in edgelist:
            self.neighbors[u].append(v)
            self.neighbors[v].append(u)
            if len(self.neighbors[u]) > 1:
                self.leaves.remove(u)
            if len(self.neighbors[v]) > 1:
                self.leaves.remove(v)

    def incident_edges(u):
        incident = []
        for v in vertices:
            if (u,v) in edgelist: 
                incident.append({u,v})
        return incident

    def neighbors(u):
        neighbors = []
        for v in vertices:
            if (u,v) in edgelist: 
                neighbors.append(v)
        return incident
    
    def add_edge(e):
        (u,v) = e
        self.neighbors[u].append(v)
        self.neighbors[v].append(u)
        if len(self.neighbors[u]) > 1:
            self.leaves.remove(u)
        if len(self.neighbors[v]) > 1:
            self.leaves.remove(v)
        
    def remove_edge(e):
        (u,v) = e
        self.neighbors[u].remove(v)
        self.neighbors[v].remove(u)
        if len(self.neighbors[u]) <= 1:
            self.leaves.append(u)
        if len(self.neighbors[v]) <= 1:
            self.leaves.append(v)

In [6]:
# union-find data structure
class disjointSet:
    def __init__(self, vs):
        #initialize the structure as disjoint 1-elt sets
        self.parent = {}
        self.size = {}
        self.rank = {}

        for v in vs:
            self.parent[v] = None
            self.size[v] = 1
            self.rank[v] = 0

    def makeSet(u):
        if x not in self.parent:
            self.parent[u] = None
            self.size[u] = 1 
            self.rank[u] = 0

    def find(self, u):
        if self.parent[u]:
            self.parent[u] = find(self.parent[u])
            return parent[u]
        else:
            return u

    def union(u,v):
        u = find(u)
        v = find(v)

        if u==v: return

        if self.size[u] < self.size[v]:
            u, v = v, u

        self.parent[v] = u
        self.size[u] = self.size[u]+self.size[v]

        return

In [4]:
class edgedict(dict):
    def __getitem__(self, key):
        if key in self:
            return super().__getitem__(key)
        return super().__getitem__(tuple(reversed(key)))

In [9]:
#need: 1) decoder graph 
decoder_graph = graph([0, 1, 2, 3], [(0, 1), (1, 2), (2, 3), (3, 0)])
#syndrome 2) list of -1 syndrome vertices
syndrome = [1, 2]
#erasure 3) list of erased edges
erasure = []
#produce modified erasure to apply peeling decoder to

#1: initialize clusers, support list, boundary lists
clusters = disjointSet(syndrome)
print(clusters.parent)
support = edgedict({e: 1 if e in erasure else 0 for e in decoder_graph.edges})
boundaries = {s: decoder_graph.neighbors[s] for s in syndrome}

#2: list all clusters with an odd number of marked vertices
L = [v for v in clusters.size if clusters.size[v] % 2 == 1]

#3 while there are odd clusters
while L != []:
    #4: initialize empty fusion list
    fusion = []
    #5 for all u in L, grom the cluster by half an edge
    for u in L:
        #if u on boundary
        for b in boundaries[u]:
            for n in decoder_graph.neighbors[b]:
                #grow each edge from u by 0.5
                #sets aren't hashable
                if support[(b,n)] < 1:
                    support[(b, n)] += 0.5
                #add new edges to fusion list
                if support[(b,n)] == 1:
                    fusion.append((b,n))
    #6: for all edges in fusion list, union the clusters if needed
    for edge in fusion:
        u = edge[0]
        v = edge[1]
        if clusters.find(u) != clusters.find(v):
            #7 for all edges in fusion list, update the boundary lists
            #####but didn't we just union the clusters?####
            root_u = clusters.find(u)
            root_v = clusters.find(v)
            if clusters.size(root_u) > clusters.size(root_v):
            #append boundary list of v to boundary list of u
                boundaries[root_u].append(boundaries[root_v])
            else:
            #append boundary list of u to boundary list of v
                boundaries[root_v].append(boundaries[root_u])
            clusters.union(u,v)
        else:
            fusion.remove(edge)

    #8 replace each u in L with find(u) (new root)
    Lnew = []
    for u in L:
        u_new = clusters.find(u)
        if u_new not in Lnew:
            Lnew.append(u_new)
            ####9: remove vertices in boundary list of u that are not boundary vertices
            for v in boundaries[u_new]:
                #if all edges have support 1, remove from boundary
                if all(support[s]==1 for s in incident_eges(v)): boundaries[u_new].remove()


    L = Lnew
    #10
    if clusters.size[clusters.find(u)] % 2 == 0:
        L.remove(u)
#11
for edge in support:
    if edge == 1: #fully-grown edge
        erasure.append(edge)
#result at this point is a list of edges making up the modified erasure

{1: None, 2: None}


KeyError: 0

In [ ]:
####12 apply peeling decoder to the erasure
#still needs work
#1: construct spanning forest of erasure
erasure_forest = graph([],erasure) #need vertex list for modified erasure
remaining_vertices = decoder_graph.vertices[1:]
r = decoder_graph.vertices[0] #pick a root
visited = {}
queue = [r]
erasure_forest.vertices.append(r)

while remaining_vertices: 
    #find a connected component
    while queue:
        u = queue.pop(0)
        neighbors = decoder_graph.neighbors(u)
        for v in neighbors:
            if v not in visited:
                visited.add(u)
                erasure_forest.edges.add_edge({u,v})
                erasure_forest.vertices.append(v) #modify add_edge to do this automatically
                queue.append(v)
                remaining_vertices.remove(v)
                

#2 initialize A
A = {}
#3: while the forest is nonempty
print(erasure_forest.edges)
while erasure_forest.edges != []:
    #pick a leaf and remove e, with pendant vertex u
    #get vertices, id pendant vertex
    u = erasure_forest.leaves[0]
    v = erasure_forest.neighbors[u][0]
    e = {u,v}
    erasure_forest.edges.remove_edge(e) #need something that updates the leaves when removing edges
    #4: if u is in the syndrome, add e to A, remove u from syndrome, and flip v in syndrome
    if u in syndrome:
        A.add(e)
        syndrome.remove(u)
        if v in syndrome:
            syndrome.remove(v)
        else:
            syndrome.add(v)    
    #(otherwise, do nothing)
#return product of Z_e for e in A
P = str()
for e in A:
    P += 'Z'+str(e)
print(P)
